In [2]:
import pandas as pd
from urllib import request
from urllib.error import HTTPError, URLError
import datetime as dt
from bs4 import BeautifulSoup as bs
import re

In [22]:
county = re.compile(r'^(\w*)\sCOUNTY.*')
town = re.compile(r'.\s?(\w*):\s(\d*).*')
deaths = re.compile(
            r'.*\w*:\s?.*(?:with)?\s(\d*)\s(?:death|fatalitie|fatality|who died)s?.*',
            re.IGNORECASE
        )
recovered = re.compile(
            r'.*(\d+)\s(?:cleared from quarantine|who recovered'\
            +r'|have recovered|recovered).*',
            re.IGNORECASE
            )

YESTERDAY = dt.datetime.now() + dt.timedelta(days = -1)
MONTHDAYYEAR = "%m-%d-%Y"
url = f"https://www.nj.com/coronavirus/{YESTERDAY.strftime('%Y')}/{YESTERDAY.strftime('%m')}/"+\
        f"where-is-the-coronavirus-in-nj-latest-map-update-on-county-by-county-cases-"+\
        f"{YESTERDAY.strftime('%B').lower()}-{YESTERDAY.strftime('%-m')}-{YESTERDAY.strftime('%Y')}.html"

In [11]:
try:
    html = bs(request.urlopen(url))
    print(f'Date: {YESTERDAY.strftime(MONTHDAYYEAR)}\nSuccessfully opened {url}.')
except HTTPError as e:
    print(f'Date: {YESTERDAY.strftime(MONTHDAYYEAR)}\nError trying to open {url}:\n{e}')
except URLError as e:
    print(f'Date: {YESTERDAY.strftime(MONTHDAYYEAR)}\nError trying to open {url}:\n{e}')

Date: 04-05-2020
Successfully opened https://www.nj.com/coronavirus/2020/04/where-is-the-coronavirus-in-nj-latest-map-update-on-county-by-county-cases-april-4-2020.html.


In [27]:
current_county = ''
towns = []

for p in html.findAll('p'):
    if county.match(p.text):
        current_county = county.match(p.text).group(1).title()
    if town.match(p.text):
        town_name = town.match(p.text).group(1)
        town_ct = int(town.match(p.text).group(2))
        death_ct = 0
        recovered_ct = 0
        if deaths.match(p.text):
            death_ct = deaths.match(p.text).group(1)
            print(p.text)
        if recovered.match(p.text):
            recovered_ct = recovered.match(p.text).group(1)
            print(p.text)
        towns.append(
                    [current_county,
                     town_name,
                     town_ct,
                     death_ct,
                     recovered_ct
                    ])

• Chesterfield: 1 (1 cleared from quarantine)
• Cinnaminson: 6 (and 1 death)
• Evesham: 31 (1 cleared from quarantine, 1 death)
• Evesham: 31 (1 cleared from quarantine, 1 death)
• Lumberton: 9 (and 1 death)
• Moorestown: 21 (3 cleared from quarantine)
• Riverside: 6 (with 1 death)
• Westampton: 9 (1 cleared from quarantine)
• Willingboro: 58 (1 cleared from quarantine)
• Wrightstown: 0 active (1 cleared from quarantine)
• Barrington: 5 and 1 death
• Camden: 34 with 1 death
• Winslow: 46 with 1 death
• Bridgeton: 4 with 1 death
• Belleville: 194, including 2 deaths
• Bloomfield: 180, including 10 deaths
• Caldwell: 24, including 1 death
• Irvington: 255, including 2 deaths
• Livingston: 145, including 10 deaths
• Maplewood: 69, including 1 death
• Montclair: 130, including 12 deaths
• Newark: 946, including 21 deaths
• Nutley: 137, including 6 deaths
• Orange: 163, including 1 death
• Roseland: 23, including 1 death
• Bayonne: 184 active cases, including approximately 12 who recovered 

In [28]:
towns

[['Bergen', 'Allendale', 21, 0, 0],
 ['Bergen', 'Alpine', 11, 0, 0],
 ['Bergen', 'Bergenfield', 252, 0, 0],
 ['Bergen', 'Bogota', 65, 0, 0],
 ['Bergen', 'Carlstadt', 28, 0, 0],
 ['Bergen', 'Closter', 25, 0, 0],
 ['Bergen', 'Cresskill', 33, 0, 0],
 ['Bergen', 'Demarest', 19, 0, 0],
 ['Bergen', 'Dumont', 91, 0, 0],
 ['Bergen', 'Edgewater', 52, 0, 0],
 ['Bergen', 'Emerson', 29, 0, 0],
 ['Bergen', 'Englewood', 229, 0, 0],
 ['Bergen', 'Fairview', 72, 0, 0],
 ['Bergen', 'Garfield', 149, 0, 0],
 ['Bergen', 'Hackensack', 304, 0, 0],
 ['Bergen', 'Haworth', 10, 0, 0],
 ['Bergen', 'Hillsdale', 41, 0, 0],
 ['Bergen', 'Leonia', 39, 0, 0],
 ['Bergen', 'Lodi', 152, 0, 0],
 ['Bergen', 'Lyndhurst', 87, 0, 0],
 ['Bergen', 'Mahwah', 79, 0, 0],
 ['Bergen', 'Maywood', 40, 0, 0],
 ['Bergen', 'Montvale', 34, 0, 0],
 ['Bergen', 'Moonachie', 23, 0, 0],
 ['Bergen', 'Northvale', 16, 0, 0],
 ['Bergen', 'Norwood', 25, 0, 0],
 ['Bergen', 'Oakland', 58, 0, 0],
 ['Bergen', 'Oradell', 48, 0, 0],
 ['Bergen', 'Paramus',